In [5]:
import numpy as np
import utils 
import pandas as pd
import os
import matplotlib.pyplot as plt
import utils

## Attempt to reduce data dimensionality with Structural Similarity Index (SSIM)
The entire code is in the file utils.py, it was not reported here because it is refered to an attempt that didn't brought to any result we wanted to keep.
The idea was to remove all the images with an SSIM of 0.95 or higher from the classes with more than 500 elements.
The results were not as expected, so we decided to keep all the images.

In [2]:
utils.create_csv_for_folders('Spettrogrammi/Target/', 8)

In [3]:
utils.create_csv_for_folders('Spettrogrammi/Non-Target/', 8)

In [ ]:
utils.merge_csv_files('Spettrogrammi/Target/dati ssim/SSIM', 'Spettrogrammi/Target/ssim_results.csv')

In [ ]:
utils.merge_csv_files('Spettrogrammi/Non-Target/dati ssim/SSIM', 'Spettrogrammi/Non-Target/ssim_results.csv')

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_non_target.describe()

In [ ]:
df_target = pd.read_csv('Spettrogrammi/Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_target.describe()

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')

df_non_target.to_csv('final_dataset/df_paths_non_target.csv', index=False)
df_target.to_csv('final_dataset/df_paths_target.csv', index=False)

# Split Dataset in Train, validation e Test

### Extracted the numbers of audios for each classes in order to avoid splitting the same audio in the different sets and removed the classes with less than 8 audios.

In [8]:
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')

In [9]:
utils.estrai_info_da_csv('Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_class_num.csv')
utils.estrai_info_da_csv('Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_class_num.csv')

In [10]:
utils.filter_csv('Spettrogrammi/Target/df_class_num.csv', 'Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_paths_filtered.csv')
utils.filter_csv('Spettrogrammi/Non-Target/df_class_num.csv', 'Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_paths_filtered.csv')

### Split the dataset in train, validation and test

In [11]:
def split_files_into_sets(input_csv_path, train_csv_path, val_csv_path, test_csv_path):
    # Imposta il seed per il generatore di numeri casuali
    np.random.seed(0)

    # Carica il file CSV in un DataFrame
    df = pd.read_csv(input_csv_path)

    # Crea una nuova colonna per il nome del file senza la parte "_resampled"
    df['File'] = df['FilePath'].apply(lambda x: x.split('_resampled')[0])

    # Inizializza una nuova colonna per il set
    df['Set'] = ''

    # Raggruppa per classe
    grouped = df.groupby('Classe')

    # Per ogni classe, assegna un set a ciascun file unico
    for name, group in grouped:
        # Ottieni un elenco di file unici
        unique_files = group['File'].unique()

        # Calcola le dimensioni dei set
        train_size = int(0.8 * len(unique_files))
        val_size = int(0.1 * len(unique_files))
        test_size = len(unique_files) - train_size - val_size

        # Assegna un set a ciascun file unico
        file_sets = ['train'] * train_size + ['val'] * val_size + ['test'] * test_size

        # Mescola l'elenco dei set di file
        np.random.shuffle(file_sets)

        # Crea un dizionario che mappa ciascun file al suo set
        file_to_set = dict(zip(unique_files, file_sets))

        # Assegna ogni riga al set del suo file
        df.loc[group.index, 'Set'] = group['File'].apply(lambda x: file_to_set[x])

    # Crea tre DataFrame separati per i set di addestramento, validazione e test
    df_train = df[df['Set'] == 'train']
    df_val = df[df['Set'] == 'val']
    df_test = df[df['Set'] == 'test']

    # Salva i DataFrame separati come nuovi CSV
    df_train.to_csv(train_csv_path, index=False)
    df_val.to_csv(val_csv_path, index=False)
    df_test.to_csv(test_csv_path, index=False)

In [12]:
split_files_into_sets('Spettrogrammi/Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_target_train.csv', 'final_dataset/validation/df_paths_target_val.csv', 'final_dataset/test/df_paths_target_test.csv')

In [13]:
split_files_into_sets('Spettrogrammi/Non-Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/validation/df_paths_non_target_val.csv', 'final_dataset/test/df_paths_non_target_test.csv')